#### Preparar Ambiente

In [1]:
import sys

In [2]:
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
!{sys.executable} -m pip install pytz

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
!{sys.executable} -m pip install tqdm

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


#### Conectar con pyathena

In [5]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [6]:
from datetime import datetime, timedelta
import numpy as np

In [7]:
from tqdm.notebook import tqdm
from time import sleep

#### Crear tablas por pais

In [8]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [9]:
pais = 'LATAM'

In [10]:
# Detalles de la tabla de ciudades
id_city = 'idn'

#### Eligiendo periodo de interes

In [11]:
# Selecionando fecha de interes
fechas = pd.date_range('09/01/2020','09/30/2020')

#### Generar tablas hogares y usuarios con adms

In [ ]:
# Selecionando fecha de interes
#fechas = pd.date_range('03/09/2020','04/30/2020')

# Generando tablas hogares y usuarios con ciudades
start_time = time.time()
for fecha in fechas:
    year = fecha.year
    mes = fecha.month
    dia = fecha.day
    codigo_year = str(year)
    codigo_mes = '0'+str(mes) if mes<10 else str(mes)
    codigo_dia = '0'+str(dia) if dia<10 else str(dia)
    codigo_fecha = codigo_year + codigo_mes + codigo_dia
    
    
    # Tabla hogares por dia
    #===========================================
    tabla_historica = f'historico_usuarios'
    tabla_hogares_dia = f'hogares_{pais}_{codigo_fecha}'
    query = f'''
            CREATE TABLE IF NOT EXISTS {tabla_hogares_dia} AS
            SELECT iso_country_code,
                   caid, 
                   lat_final,
                   long_final
            FROM {tabla_historica}                            
            WHERE month={mes} AND day={dia} AND lat_final IS NOT NULL
            GROUP BY iso_country_code,
                     caid, 
                     lat_final,
                     long_final
    '''
    cursor.execute(query)
    time.sleep(20)
    #
    cursor.close()
    #print(f"Table {tabla_hogares_dia}: DONE ")
    
    
    # Asignando ciudades a tabla hogares por dia
    #===========================================
    table_hogares_cities_dia = f'hogares_cities_{pais}_{codigo_fecha}'
    query = f'''
                    CREATE TABLE {table_hogares_cities_dia} AS
                    SELECT iso_country_code,
                            caid,
                            lat_final,
                            long_final,
                            {id_city} as id_city
                    FROM {tabla_hogares_dia}
                    CROSS JOIN {pais}_cities
                    WHERE ST_CONTAINS (ST_GeomFromLegacyBinary({pais}_cities.boundary), ST_POINT({tabla_hogares_dia}.long_final, {tabla_hogares_dia}.lat_final))
    '''
    cursor.execute(query)
    time.sleep(20)
    #
    cursor.close()
    #print(f"Table {table_hogares_cities_dia}: DONE ")
    
    
    # Tabla usuarios por dia
    #===========================================
    table_usuarios_cities_dia = f'usuarios_cities_{pais}_{codigo_fecha}'
    query = f'''
        CREATE TABLE {table_usuarios_cities_dia} AS
        SELECT  g.id_city,
                h.caid,
                h.distancia_recorrida,
                h.lat_final,
                h.long_final,
                h.n_obs,
                h.n_obs_noche,
                h.n_obs_madrugada,
                h.day,
                h.month,
                h.year,
                h.iso_country_code
        FROM (SELECT caid,
                     id_city
              FROM {table_hogares_cities_dia}) as g INNER JOIN (
                SELECT *
                FROM historico_usuarios
                WHERE month={mes} AND day={dia}) as h
            ON h.caid=g.caid
    '''
    cursor.execute(query)
    time.sleep(20)
    #
    cursor.close()
    #print(f"Table {table_usuarios_cities_dia}: DONE ")

    
    # Tabla pings con ciudades de hogar por dia
    #===========================================
    tabla_pings_citiesh_dia = f'pings_citiesh_{pais}_{codigo_fecha}'
    query = f'''
        CREATE TABLE {tabla_pings_citiesh_dia} AS
        SELECT  g.id_city,
                h.utc_timestamp,
                h.latitude,
                h.longitude,
                h.caid,
                h.id_type,
                h.hour_of_day,
                h.day,
                h.month,
                h.year,
                h.iso_country_code
        FROM (SELECT caid,
                     id_city
              FROM {table_hogares_cities_dia}) as g INNER JOIN (
                SELECT *
                FROM historico_pings
                WHERE month={mes} AND day={dia}) as h
            ON h.caid=g.caid
    '''
    cursor.execute(query)
    #print(f"Table {tabla_pings_citiesh_dia}: DONE ")
    
    
    print(f"FECHA {fecha} DONE ")

print((time.time()-start_time)/60)    

FECHA 2020-09-01 00:00:00 DONE 
FECHA 2020-09-02 00:00:00 DONE 
FECHA 2020-09-03 00:00:00 DONE 
FECHA 2020-09-04 00:00:00 DONE 
FECHA 2020-09-05 00:00:00 DONE 
FECHA 2020-09-06 00:00:00 DONE 
FECHA 2020-09-07 00:00:00 DONE 
FECHA 2020-09-08 00:00:00 DONE 
FECHA 2020-09-09 00:00:00 DONE 
FECHA 2020-09-10 00:00:00 DONE 
FECHA 2020-09-11 00:00:00 DONE 
FECHA 2020-09-12 00:00:00 DONE 
FECHA 2020-09-13 00:00:00 DONE 
FECHA 2020-09-14 00:00:00 DONE 
FECHA 2020-09-15 00:00:00 DONE 
FECHA 2020-09-16 00:00:00 DONE 
FECHA 2020-09-17 00:00:00 DONE 
FECHA 2020-09-18 00:00:00 DONE 
FECHA 2020-09-19 00:00:00 DONE 
FECHA 2020-09-20 00:00:00 DONE 
FECHA 2020-09-21 00:00:00 DONE 
FECHA 2020-09-23 00:00:00 DONE 
FECHA 2020-09-24 00:00:00 DONE 
FECHA 2020-09-25 00:00:00 DONE 
FECHA 2020-09-26 00:00:00 DONE 
FECHA 2020-09-27 00:00:00 DONE 
FECHA 2020-09-28 00:00:00 DONE 
FECHA 2020-09-29 00:00:00 DONE 
FECHA 2020-09-30 00:00:00 DONE 
66.09640263319015


#### Generar tablas pings con adms

In [ ]:
# Selecionando fecha de interes
#fechas = pd.date_range('02/04/2020','04/30/2020')

# Generando tabla pings con ciudades
start_time = time.time()
for fecha in fechas:
    year = fecha.year
    mes = fecha.month
    dia = fecha.day
    codigo_year = str(year)
    codigo_mes = '0'+str(mes) if mes<10 else str(mes)
    codigo_dia = '0'+str(dia) if dia<10 else str(dia)
    codigo_fecha = codigo_year + codigo_mes + codigo_dia

    # Asignando ciudades a tabla pings por dia
    #===========================================
    tabla_pings_citiesh_dia = f'pings_citiesh_{pais}_{codigo_fecha}'
    tabla_pings_citiesp_dia = f'pings_citiesp_{pais}_{codigo_fecha}'
    query = f'''
                CREATE TABLE {tabla_pings_citiesp_dia} AS
                SELECT utc_timestamp,
                    caid,
                    id_type,
                    hour_of_day,
                    day,
                    month,
                    year,
                    iso_country_code,
                    {id_city} as id_cityp
                FROM {tabla_pings_citiesh_dia}
                CROSS JOIN {pais}_cities
                WHERE ST_CONTAINS (ST_GeomFromLegacyBinary({pais}_cities.boundary), ST_POINT({tabla_pings_citiesh_dia}.longitude, {tabla_pings_citiesh_dia}.latitude))
    '''
    cursor.execute(query)
    
    print(f"FECHA {fecha} DONE ")

print((time.time()-start_time)/60)         

FECHA 2020-09-01 00:00:00 DONE 
FECHA 2020-09-02 00:00:00 DONE 
FECHA 2020-09-03 00:00:00 DONE 
FECHA 2020-09-04 00:00:00 DONE 
FECHA 2020-09-05 00:00:00 DONE 
FECHA 2020-09-06 00:00:00 DONE 
FECHA 2020-09-07 00:00:00 DONE 
FECHA 2020-09-08 00:00:00 DONE 
FECHA 2020-09-09 00:00:00 DONE 
FECHA 2020-09-10 00:00:00 DONE 
FECHA 2020-09-11 00:00:00 DONE 
FECHA 2020-09-12 00:00:00 DONE 
FECHA 2020-09-13 00:00:00 DONE 
FECHA 2020-09-14 00:00:00 DONE 
FECHA 2020-09-15 00:00:00 DONE 
FECHA 2020-09-16 00:00:00 DONE 
FECHA 2020-09-17 00:00:00 DONE 
FECHA 2020-09-18 00:00:00 DONE 
FECHA 2020-09-19 00:00:00 DONE 
FECHA 2020-09-20 00:00:00 DONE 
FECHA 2020-09-21 00:00:00 DONE 
FECHA 2020-09-22 00:00:00 DONE 
FECHA 2020-09-23 00:00:00 DONE 
FECHA 2020-09-24 00:00:00 DONE 
FECHA 2020-09-25 00:00:00 DONE 
FECHA 2020-09-26 00:00:00 DONE 
FECHA 2020-09-27 00:00:00 DONE 
FECHA 2020-09-28 00:00:00 DONE 
FECHA 2020-09-29 00:00:00 DONE 
FECHA 2020-09-30 00:00:00 DONE 
15.217806343237559


#### Generar tablas matriz origen-destino

In [ ]:
# Selecionando fecha de interes
#fechas = pd.date_range('02/01/2020','04/30/2020')

# Generando matriz origen-destino por dia
start_time = time.time()
for fecha in fechas:
    year = fecha.year
    mes = fecha.month
    dia = fecha.day
    codigo_year = str(year)
    codigo_mes = '0'+str(mes) if mes<10 else str(mes)
    codigo_dia = '0'+str(dia) if dia<10 else str(dia)
    codigo_fecha = codigo_year + codigo_mes + codigo_dia
    
    
    # Generando matriz origen-destino por dia
    #===========================================
    tabla_usuarios_cities_dia = f'usuarios_cities_{pais}_{codigo_fecha}'
    tabla_pings_citiesp_dia = f'pings_citiesp_{pais}_{codigo_fecha}'
    
    tabla_matrizOD_cities_dia = f'matriz_od_cities_{pais}_{codigo_fecha}'
    query_matriz_OD = f'''CREATE TABLE {tabla_matrizOD_cities_dia} AS
                          SELECT year, 
                                month, 
                                day, 
                                id_city, 
                                id_cityh, 
                                COUNT(*) AS n
                            FROM (SELECT year, 
                                        month, 
                                        day, 
                                        LHS.caid AS caid, 
                                        id_city, 
                                        id_cityh, 
                                        n
                                    FROM (SELECT year, 
                                                month, 
                                                day, 
                                                caid, 
                                                id_city, 
                                                n
                                            FROM (SELECT year, 
                                                        month, 
                                                        day, 
                                                        caid, 
                                                        id_city, 
                                                        n, 
                                                        MAX(n) 
                                                        OVER (PARTITION BY year, 
                                                                        month, 
                                                                        day, 
                                                                        caid) AS q01
                                                FROM (SELECT year, 
                                                            month, 
                                                            day, 
                                                            caid, 
                                                            id_cityp as id_city, 
                                                            COUNT(*) AS n
                                                        FROM {tabla_pings_citiesp_dia}
                                                        GROUP BY year, 
                                                                month, 
                                                                day, 
                                                                caid, 
                                                                id_cityp)
                                                  )
                                            WHERE (n = q01)) AS LHS
                                            LEFT JOIN (SELECT caid, 
                                                            id_city as id_cityh 
                                                        FROM {tabla_usuarios_cities_dia}) AS RHS
                                            ON (LHS.caid = RHS.caid)
                                  )
                        GROUP BY year, 
                                month, 
                                day, 
                                id_city, 
                                id_cityh 
                        ORDER BY year ASC, 
                                month ASC, 
                                day ASC, 
                                n DESC
    '''   
    cursor.execute(query_matriz_OD)

    print(f"FECHA {fecha} DONE ")

print((time.time()-start_time)/60) 

FECHA 2020-09-01 00:00:00 DONE 
FECHA 2020-09-02 00:00:00 DONE 
FECHA 2020-09-03 00:00:00 DONE 
FECHA 2020-09-04 00:00:00 DONE 
FECHA 2020-09-05 00:00:00 DONE 
FECHA 2020-09-06 00:00:00 DONE 
FECHA 2020-09-07 00:00:00 DONE 
FECHA 2020-09-08 00:00:00 DONE 
FECHA 2020-09-09 00:00:00 DONE 
FECHA 2020-09-10 00:00:00 DONE 
FECHA 2020-09-11 00:00:00 DONE 
FECHA 2020-09-12 00:00:00 DONE 
FECHA 2020-09-13 00:00:00 DONE 
FECHA 2020-09-14 00:00:00 DONE 
FECHA 2020-09-15 00:00:00 DONE 
FECHA 2020-09-16 00:00:00 DONE 
FECHA 2020-09-17 00:00:00 DONE 
FECHA 2020-09-18 00:00:00 DONE 
FECHA 2020-09-19 00:00:00 DONE 
FECHA 2020-09-20 00:00:00 DONE 
FECHA 2020-09-21 00:00:00 DONE 
FECHA 2020-09-22 00:00:00 DONE 
FECHA 2020-09-23 00:00:00 DONE 
FECHA 2020-09-24 00:00:00 DONE 
FECHA 2020-09-25 00:00:00 DONE 
FECHA 2020-09-26 00:00:00 DONE 
FECHA 2020-09-27 00:00:00 DONE 
FECHA 2020-09-28 00:00:00 DONE 
FECHA 2020-09-29 00:00:00 DONE 
FECHA 2020-09-30 00:00:00 DONE 
4.227080512046814


#### Eliminando tablas temporales

In [ ]:
#fechas = pd.date_range('02/01/2020','04/30/2020')
#pais = 'BR'

# Eliminando tablas temporales
start_time = time.time()
try:
    for fecha in fechas:
        year = fecha.year
        mes = fecha.month
        dia = fecha.day
        codigo_year = str(year)
        codigo_mes = '0'+str(mes) if mes<10 else str(mes)
        codigo_dia = '0'+str(dia) if dia<10 else str(dia)
        codigo_fecha = codigo_year + codigo_mes + codigo_dia
    
        tabla_datos_pais = f'hogares_{pais}_{codigo_fecha}'
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)

        tabla_datos_pais = f'hogares_cities_{pais}_{codigo_fecha}'
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)

        tabla_datos_pais = f'usuarios_cities_{pais}_{codigo_fecha}'
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)

        tabla_datos_pais = f'pings_citiesh_{pais}_{codigo_fecha}'
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)

        tabla_datos_pais = f'pings_citiesp_{pais}_{codigo_fecha}'
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)

                
except:

    print(f" Eliminado ")
    
print((time.time()-start_time)/60)     

3.8977738777796427


#fechas = pd.date_range('09/01/2020','09/30/2020')
#pais = 'BR'
start_time = time.time()
try:
    for fecha in fechas:
        year = fecha.year
        mes = fecha.month
        dia = fecha.day
        codigo_year = str(year)
        codigo_mes = '0'+str(mes) if mes<10 else str(mes)
        codigo_dia = '0'+str(dia) if dia<10 else str(dia)
        codigo_fecha = codigo_year + codigo_mes + codigo_dia
    
        tabla_datos_pais = f'matriz_od_cities_{pais}_{codigo_fecha}'
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)
                
except:

    print(f" Eliminado ")
    
print((time.time()-start_time)/60)     